<a href="https://colab.research.google.com/github/OlajideFemi/GeneralContent/blob/master/MailMerge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install gspread google-auth google-api-python-client jinja2


In [3]:
# 1. Setup credentials and authorize access to Google Sheets, Drive, and Gmail
sheet = gspread.open_by_key(1VFsnz3_4dOcHq7Q0JklwwP2hQwLw_Sb8dm7xfikEqSE).worksheet("Sheet1")
rows = sheet.get_all_records()

for row_idx, row in enumerate(rows, start=2):  # assuming row 1 = headers
    if row["Email Sent"]:
        continue

    recipient = row["Recipient"]
    file_id = row["Attachment Sheet ID"]

    # 2. Download the attachment sheet as PDF
    file_data = download_sheet_as_pdf(file_id)

    # 3. Get Gmail draft and merge placeholders (e.g. {{First Name}})
    subject, body = render_gmail_draft_with_row(row)

    # 4. Send the email via Gmail API with attachment
    send_email_with_attachment(recipient, subject, body, file_data)

    # 5. Update the "Email Sent" column
    sheet.update_cell(row_idx, sent_col_index, timestamp_now())


SyntaxError: invalid decimal literal (ipython-input-3-3582268046.py, line 2)

In [4]:
!pip install --upgrade --quiet google-api-python-client google-auth-httplib2 google-auth-oauthlib gspread xlsxwriter

from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth.transport.requests import Request
from google.oauth2 import service_account

from googleapiclient.discovery import build
import base64, os, io
from email.message import EmailMessage
import xlsxwriter

# Authorize access to Google Sheets
#gc = gspread.authorize(auth.get_user_credentials())


In [ ]:
!pip install --upgrade --quiet google-api-python-client google-auth-httplib2 google-auth-oauthlib gspread xlsxwriter

from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth.transport.requests import Request
from google.oauth2 import service_account
from googleapiclient.discovery import build
import base64, os, io
from email.message import EmailMessage
import xlsxwriter
from datetime import datetime

# Initialize services
gc = gspread.authorize(auth.get_user_credentials())
service = build('gmail', 'v1')

def get_sheet_data(sheet_url):
    """Get data from Google Sheet"""
    sh = gc.open_by_url(sheet_url)
    worksheet = sh.sheet1
    data = worksheet.get_all_records()
    headers = worksheet.row_values(1)
    return data, headers

def create_excel_attachment(data, filename):
    """Create Excel attachment from data"""
    output = io.BytesIO()
    workbook = xlsxwriter.Workbook(output)
    worksheet = workbook.add_worksheet()

    # Write headers
    headers = list(data[0].keys())
    for col, header in enumerate(headers):
        worksheet.write(0, col, header)

    # Write data
    for row, item in enumerate(data, start=1):
        for col, key in enumerate(headers):
            worksheet.write(row, col, str(item.get(key, '')))

    workbook.close()
    output.seek(0)
    return output.getvalue()

def send_email(recipient, subject, message_text, attachment_data=None, attachment_name='data.xlsx'):
    """Send email with optional attachment"""
    message = EmailMessage()
    message['To'] = recipient
    message['Subject'] = subject
    message.set_content(message_text)

    if attachment_data:
        message.add_attachment(attachment_data, maintype='application',
                             subtype='vnd.openxmlformats-officedocument.spreadsheetml.sheet',
                             filename=attachment_name)

    encoded_message = base64.urlsafe_b64encode(message.as_bytes()).decode()
    body = {'raw': encoded_message}

    try:
        message = service.users().messages().send(userId='me', body=body).execute()
        return message
    except Exception as e:
        print(f'Error sending email to {recipient}: {str(e)}')
        return None

def run_mail_merge(sheet_url, subject_template, message_template):
    """Main mail merge function"""
    data, headers = get_sheet_data(sheet_url)

    for row in data:
        if not row.get('Email Sent', ''):  # Only send if not already sent
            # Customize message
            subject = subject_template.format(**row)
            message = message_template.format(**row)

            # Create attachment
            attachment_data = create_excel_attachment([row], f'data_{row["Recipient"]}.xlsx')

            # Send email
            result = send_email(
                recipient=row['Recipient'],
                subject=subject,
                message_text=message,
                attachment_data=attachment_data
            )

            if result:
                print(f"Email sent to {row['Recipient']}")
                # Mark as sent in the sheet (would need write permissions)
                # worksheet.update_cell(row_index, email_sent_col, str(datetime.now()))
            else:
                print(f"Failed to send to {row['Recipient']}")

# Example usage
SHEET_URL = "YOUR_SHEET_URL_HERE"
SUBJECT_TEMPLATE = "Your report for {Name}"
MESSAGE_TEMPLATE = """Hello {Name},

Here's your personalized report.

Regards,
The Team"""

run_mail_merge(SHEET_URL, SUBJECT_TEMPLATE, MESSAGE_TEMPLATE)

In [5]:
!pip install --upgrade --quiet google-api-python-client google-auth-httplib2 google-auth-oauthlib gspread xlsxwriter

from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth.transport.requests import Request
from google.oauth2 import service_account
from googleapiclient.discovery import build
import base64, os, io
from email.message import EmailMessage
import xlsxwriter
from datetime import datetime

# Initialize services
gc = gspread.authorize(auth.get_user_credentials())
service = build('gmail', 'v1')

def get_sheet_data(sheet_url):
    """Get data from Google Sheet"""
    sh = gc.open_by_url(sheet_url)
    worksheet = sh.sheet1
    data = worksheet.get_all_records()
    headers = worksheet.row_values(1)
    return data, headers

def create_excel_attachment(data, filename):
    """Create Excel attachment from data"""
    output = io.BytesIO()
    workbook = xlsxwriter.Workbook(output)
    worksheet = workbook.add_worksheet()

    # Write headers
    headers = list(data[0].keys())
    for col, header in enumerate(headers):
        worksheet.write(0, col, header)

    # Write data
    for row, item in enumerate(data, start=1):
        for col, key in enumerate(headers):
            worksheet.write(row, col, str(item.get(key, '')))

    workbook.close()
    output.seek(0)
    return output.getvalue()

def send_email(recipient, subject, message_text, attachment_data=None, attachment_name='data.xlsx'):
    """Send email with optional attachment"""
    message = EmailMessage()
    message['To'] = recipient
    message['Subject'] = subject
    message.set_content(message_text)

    if attachment_data:
        message.add_attachment(attachment_data, maintype='application',
                             subtype='vnd.openxmlformats-officedocument.spreadsheetml.sheet',
                             filename=attachment_name)

    encoded_message = base64.urlsafe_b64encode(message.as_bytes()).decode()
    body = {'raw': encoded_message}

    try:
        message = service.users().messages().send(userId='me', body=body).execute()
        return message
    except Exception as e:
        print(f'Error sending email to {recipient}: {str(e)}')
        return None

def run_mail_merge(sheet_url, subject_template, message_template):
    """Main mail merge function"""
    data, headers = get_sheet_data(sheet_url)

    for row in data:
        if not row.get('Email Sent', ''):  # Only send if not already sent
            # Customize message
            subject = subject_template.format(**row)
            message = message_template.format(**row)

            # Create attachment
            attachment_data = create_excel_attachment([row], f'data_{row["Recipient"]}.xlsx')

            # Send email
            result = send_email(
                recipient=row['Recipient'],
                subject=subject,
                message_text=message,
                attachment_data=attachment_data
            )

            if result:
                print(f"Email sent to {row['Recipient']}")
                # Mark as sent in the sheet (would need write permissions)
                # worksheet.update_cell(row_index, email_sent_col, str(datetime.now()))
            else:
                print(f"Failed to send to {row['Recipient']}")

# Example usage
SHEET_URL = "YOUR_SHEET_URL_HERE"
SUBJECT_TEMPLATE = "Your report for {Name}"
MESSAGE_TEMPLATE = """Hello {Name},

Here's your personalized report.

Regards,
The Team"""

run_mail_merge(SHEET_URL, SUBJECT_TEMPLATE, MESSAGE_TEMPLATE)

AttributeError: module 'google.colab.auth' has no attribute 'get_user_credentials'